## Final Project Submission

Please fill out:
* Student name: Elizabeth Fawcett
* Student pace: part time
* Scheduled project review date/time: 
* Instructor name: Brandon Lewis
* Blog post URL: 


## Methodology

To evaluate the King's country dataset, I have chosen to use the following steps:
- Business Understanding
- Data Understanding and Data Prep
 - Obtain Data
 - Scrub Data
 - Explore Data
- Modeling
- Interpret
- Evaluation
- Deployment of Model

This is a combination of the OSEMN method explained at:[insert link here] and the CRISP-DM method explained at:[insert line here]. I found that the OSEMN method explained the data science process, but did not provide a framework for contextualization of the model or original data; whereas, the CRISP-DM method provided the business understanding and deployment steps to handle defining the scope of a project. I found the descriptors in the CRISP-DM method for projects did not specify obtaining or cleaning the data, which is where the OSEMN method shines. Both methods have strengths and weakness; as a result, I decided to pull the details from both to enrich my project building experience and to always keep the client in mind by expanding on the steps of both methodologies.

## Business Understanding

To develop an understanding of the nature of the business of the housing market, I'm referencing my own experience apartment shopping, the things I learned about what's important to me as a consumer of housing, an article written by one of the top real estate leaders in the northeast about common seller's questions, and the information provided in the dataset. 

In my own experience, when looking for an apartment, as a home buyer, I am looking for livability and the condition of the home. Will I need to invest to ensure that the place meets my personal standard? Do the features available meet my needs? For instance, in the past, a one bedroom apartment was sufficent. However, now I'm looking to use a bedroom as an office, so I am looking for two bedroom apartments, but one bathroom is still okay. To conclude, as a consumer I look at the quality of my investment and then determine if the space meets my needs.

From a real estate perspective, some of the most frequently asked questions by a home seller include the following. What does the market look like right now? At what point in the year should I sell? How can I improve my home to sell? What price should I set? What is the actual value of my home? insert [reference] here. It seems as if a seller is trying to pinpoint where they are on the sliding scale of value for their surrounding area, and maximizing profit based on this scale.

The data provides both categorical and continuous data points that fall into a few different categories. The continuous data deals with square footage, time, and location; whereas the categorical data includes aspects of the home that cannot change, one of which is whether or not the property is waterfront, and opinions about the home. This is a nice mixture of data that buyers and sellers would be interested in. There are some data points that include information about the surrounding neighborhood to develop the type of sliding scale of home value as well as the condition of the home and included features.

## Data Understanding and Data Prep

### Obtain Data

The dataset used for this exploration is located in this repository under the filename: kc_house_data.csv.

In [259]:
# Your code here - remember to use markdown cells for comments as well!
# import data set from repository
import pandas as pd
df = pd.read_csv('kc_house_data.csv', index_col=0)
df.head()

,date,price,bedrooms,bathrooms,sqft_living,sqft_lot,floors,waterfront,view,condition,grade,sqft_above,sqft_basement,yr_built,yr_renovated,zipcode,lat,long,sqft_living15,sqft_lot15
id,,,,,,,,,,,,,,,,,,,,
7129300520,10/13/2014,221900.0,3,1.00,1180,5650,1.0,NaN,0.0,3,7,1180,0.0,1955,0.0,98178,47.5112,-122.257,1340,5650
6414100192,12/9/2014,538000.0,3,2.25,2570,7242,2.0,0.0,0.0,3,7,2170,400.0,1951,1991.0,98125,47.7210,-122.319,1690,7639
5631500400,2/25/2015,180000.0,2,1.00,770,10000,1.0,0.0,0.0,3,6,770,0.0,1933,NaN,98028,47.7379,-122.233,2720,8062
2487200875,12/9/2014,604000.0,4,3.00,1960,5000,1.0,0.0,0.0,5,7,1050,910.0,1965,0.0,98136,47.5208,-122.393,1360,5000
1954400510,2/18/2015,510000.0,3,2.00,1680,8080,1.0,0.0,0.0,3,8,1680,0.0,1987,0.0,98074,47.6168,-122.045,1800,7503


Primary observations about the data in this king county housing data include:
- There are n/a's in the data set
- Even though most of the data is numerical, a lot of the numbers are categorical, like number of bedrooms
- There's something funky going on with the column names
- The years in yr_renovated have floats, but they need to be integers, and this is the case with the floors column as well

### Scrub Data

The following scrubbing techniques were applied to the original data:
- Manage n/a data types
- Delete duplicate rows
- Sort into categorical and continuous data
- Correct data types

### Manage n/a data types

In [260]:
# categorical nans identification
print(len(df))
df.isna().sum()

21597


date                0
price               0
bedrooms            0
bathrooms           0
sqft_living         0
sqft_lot            0
floors              0
waterfront       2376
view               63
condition           0
grade               0
sqft_above          0
sqft_basement       0
yr_built            0
yr_renovated     3842
zipcode             0
lat                 0
long                0
sqft_living15       0
sqft_lot15          0
dtype: int64

The waterfront column is missing a lot values, what percentage of the dataset do these missing values constitute? The view column is missing far less, but what percentage of the data does it represent? This data set answers how many views does the property have prior to selling, this could be useful information for the seller in changing their price point as number of views increases.

In [261]:
# how much of the data is missing?
print('% of waterfront missing data: ' + str(df['waterfront'].isna().sum() / len(df['waterfront'])))
print('% of view missing data: ' + str(df['view'].isna().sum() / len(df['view'])))
print('% of yr_renovated data: ' + str(df['yr_renovated'].isna().sum() / len(df['yr_renovated'])))

% of waterfront missing data: 0.11001527989998611
% of view missing data: 0.0029170718155299346
% of yr_renovated data: 0.17789507802009538


Since the missing waterfront and yr_renovated data is more than 10% of the dataset, I'll create a new category 'unknown'.  Since the missing view data is less than 1% of the dataset, I'll remove those rows.

In [262]:
# managing categorical nans
# fill waterfront nans with unknown
waterfront_clean = pd.DataFrame(df.loc[:,'waterfront']).fillna(value='unknown')
yr_renovated_clean = pd.DataFrame(df.loc[:,'yr_renovated']).fillna(value='unknown')
df_clean = df_clean.drop(labels='waterfront', axis=1)
df_clean = df_clean.drop(labels='yr_renovated', axis=1)
df_clean = pd.merge(df_clean, waterfront_clean, left_index=True, right_index=True)
df_clean = pd.merge(df_clean, yr_renovated_clean, left_index=True, right_index=True)
# df_clean.head()

# delete nans rows in view
df_clean = df_clean.dropna()
print(len(df_clean))
df_clean.isna().sum()

23553


date             0
price            0
bedrooms         0
bathrooms        0
sqft_living      0
sqft_lot         0
floors           0
view             0
condition        0
grade            0
sqft_above       0
sqft_basement    0
yr_built         0
zipcode          0
lat              0
long             0
sqft_living15    0
sqft_lot15       0
waterfront       0
yr_renovated     0
dtype: int64

### Delete duplicate rows

In [263]:
print('duplicated rows: ' + str(df_clean.duplicated().sum()))

duplicated rows: 1794


In [265]:
df_clean.drop_duplicates(inplace=True)
print('duplicated rows: ' + str(df_clean.duplicated().sum()))
print(len(df_clean))
df_clean.head()

duplicated rows: 0
21759


,date,price,bedrooms,bathrooms,sqft_living,sqft_lot,floors,view,condition,grade,sqft_above,sqft_basement,yr_built,zipcode,lat,long,sqft_living15,sqft_lot15,waterfront,yr_renovated
id,,,,,,,,,,,,,,,,,,,,
1000102,9/16/2014,280000.0,6,3.00,2400,9373,2.0,0.0,3,7,2400,0.0,1991,98002,47.3262,-122.214,2060,7316,unknown,0
1000102,9/16/2014,280000.0,6,3.00,2400,9373,2.0,0.0,3,7,2400,0.0,1991,98002,47.3262,-122.214,2060,7316,0,0
1000102,4/22/2015,300000.0,6,3.00,2400,9373,2.0,0.0,3,7,2400,0.0,1991,98002,47.3262,-122.214,2060,7316,unknown,0
1000102,4/22/2015,300000.0,6,3.00,2400,9373,2.0,0.0,3,7,2400,0.0,1991,98002,47.3262,-122.214,2060,7316,0,0
1200019,5/8/2014,647500.0,4,1.75,2060,26036,1.0,0.0,4,8,1160,900.0,1947,98166,47.4444,-122.351,2590,21891,unknown,0


### Sort into categorical and continuous data

In [266]:
categorical = df_clean.loc[:,['zipcode', 'yr_built','bedrooms', 'bathrooms', 'floors', 'waterfront', 'yr_renovated', 'view', 'condition', 'grade']]
# categorical.head()
continuous = df_clean.loc[:, ['lat', 'long', 'sqft_lot', 'sqft_living', 'sqft_above', 'sqft_basement', 'date', 'sqft_living15', 'sqft_lot15']]
# continuous.head()
target = pd.DataFrame(df_clean.loc[:, 'price'])
# target.head()

### Correct data types

In [267]:
categorical.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 21759 entries, 1000102 to 9900000190
Data columns (total 10 columns):
zipcode         21759 non-null int64
yr_built        21759 non-null int64
bedrooms        21759 non-null int64
bathrooms       21759 non-null float64
floors          21759 non-null float64
waterfront      21759 non-null object
yr_renovated    21759 non-null object
view            21759 non-null float64
condition       21759 non-null int64
grade           21759 non-null int64
dtypes: float64(3), int64(5), object(2)
memory usage: 1.8+ MB


What do the values look like in each of these categories?

In [268]:
print(categorical.zipcode.unique())
print(categorical.yr_built.unique())
print(categorical.bedrooms.unique())
print(categorical.bathrooms.unique())
print(categorical.floors.unique())
print(categorical.waterfront.unique())
print(categorical.yr_renovated.unique())
print(categorical.view.unique())
print(categorical.condition.unique())
print(categorical.grade.unique())

[98002 98166 98168 98144 98178 98108 98032 98055 98118 98122 98115 98007
 98034 98052 98042 98040 98136 98126 98146 98188 98059 98109 98116 98103
 98006 98075 98024 98155 98003 98023 98117 98177 98019 98028 98092 98022
 98070 98038 98107 98106 98065 98053 98072 98077 98133 98001 98056 98045
 98033 98011 98014 98199 98008 98031 98004 98074 98030 98027 98029 98058
 98010 98005 98198 98148 98112 98125 98105 98102 98119 98039]
[1991 1947 1952 1930 1951 1960 2005 1945 1924 1925 1902 2014 1901 2007
 1941 1998 1994 1993 1997 1995 1996 2003 1913 1907 1915 1908 1985 1918
 1921 1949 1914 1975 1978 1958 1956 1954 1959 1977 1955 1939 2013 1942
 1950 1929 2001 1928 1966 1965 2012 1920 1957 2008 1934 1983 1962 2004
 2010 1963 1940 1988 1912 1999 1968 2002 2006 2009 1976 1946 1981 1984
 1986 1987 1989 1990 1948 1979 1980 1937 1970 1927 1923 1909 1916 1943
 2000 1971 1974 1911 1961 1967 1922 1910 2011 1944 1931 1905 1936 1917
 1992 1964 1919 1973 1903 1982 1926 1972 1953 1969 2015 1932 1904 1900
 1933

The output above for zipcode, yr_built, and bedrooms look good, and as expected.
The output for bathrooms looks strange, it's a float64, but some of the values are whole numbers, change the values here to have 2 decimal places.
The output for floors is similar to bathrooms, change the values to have one decimal place.
View needs to be changed to an int64 value.
Condition and grade look good and as expected.

Since waterfront and yr_renovated are already objects, chaning their number formatting needs to happen while categorizing.
Waterfront doesn't need decimal places at all, change to an int64.
Yr_renovated doesn't need decimal places either.

In [269]:
# bathrooms update for all values to have decimal places
bathrooms_clean = pd.DataFrame(categorical.bathrooms.astype(str))
categorical = categorical.drop(labels='bathrooms', axis=1)
categorical = pd.merge(categorical, bathrooms_clean, how='outer', left_index=True, right_index=True)
new_columns = ['zipcode', 'yr_built', 'bedrooms', 'floors', 'waterfront', 'yr_renovated', 'view', 'condition', 'grade', 'bathrooms']
categorical.columns = new_columns
print(categorical.info())
print(categorical.bathrooms.unique())
print('duplicated rows: ' + str(df_clean.duplicated().sum()))

<class 'pandas.core.frame.DataFrame'>
Int64Index: 23633 entries, 1000102 to 9900000190
Data columns (total 10 columns):
zipcode         23633 non-null int64
yr_built        23633 non-null int64
bedrooms        23633 non-null int64
floors          23633 non-null float64
waterfront      23633 non-null object
yr_renovated    23633 non-null object
view            23633 non-null float64
condition       23633 non-null int64
grade           23633 non-null int64
bathrooms       23633 non-null object
dtypes: float64(2), int64(5), object(3)
memory usage: 2.0+ MB
None
['3.0' '1.75' '1.0' '2.0' '2.75' '1.5' '2.5' '2.25' '3.25' '3.5' '4.5'
 '3.75' '0.75' '4.0' '5.0' '5.25' '4.25' '7.5' '6.5' '5.5' '4.75' '6.0'
 '1.25' '8.0' '5.75' '6.75' '6.25' '0.5' '7.75']
duplicated rows: 0


In [270]:
# floors update for all values to have decimal places
floors_clean = pd.DataFrame(categorical.floors.astype(str))
categorical = categorical.drop(labels='floors', axis=1)
categorical = pd.merge(categorical, floors_clean, how='left', left_index=True, right_index=True)
new_columns = ['zipcode', 'yr_built', 'bedrooms', 'waterfront', 'yr_renovated', 'view', 'condition', 'grade', 'bathrooms', 'floors']
categorical.columns = new_columns
print(categorical.info())
print(categorical.floors.unique())
print('duplicated rows: ' + str(df_clean.duplicated().sum()))

<class 'pandas.core.frame.DataFrame'>
Int64Index: 114317 entries, 1000102 to 9900000190
Data columns (total 10 columns):
zipcode         114317 non-null int64
yr_built        114317 non-null int64
bedrooms        114317 non-null int64
waterfront      114317 non-null object
yr_renovated    114317 non-null object
view            114317 non-null float64
condition       114317 non-null int64
grade           114317 non-null int64
bathrooms       114317 non-null object
floors          114317 non-null object
dtypes: float64(1), int64(5), object(4)
memory usage: 9.6+ MB
None
['2.0' '1.0' '1.5' '3.0' '2.5' '3.5']
duplicated rows: 0


In [193]:
# waterfront update to int64 # needs to be done after conversion into categories?
waterfront_int = pd.DataFrame(categorical.waterfront.astype(int))
categorical = categorical.drop(labels='waterfront', axis=1)
categorical = pd.merge(categorical, waterfront_int, left_index=True, right_index=True)
new_columns = ['zipcode', 'yr_built', 'bedrooms', 'yr_renovated', 'view', 'condition', 'grade', 'bathrooms', 'floors', 'waterfront']
categorical.columns = new_columns
print(categorical.info())
print(categorical.waterfront.unique())

ValueError: invalid literal for int() with base 10: 'unknown'

#### Categorization time!

For each of the columns in the categorical dataframe, a change to categorical data is needed.

In [ ]:
# zipcode categories
zip_cate = pd.DataFrame(categorical.loc[:,'zipcode'].astype('category'))
categorical = categorical.drop(labels='zipcode', axis=1)
categorical = pd.merge(categorical, zip_cate, left_index=True, right_index=True)
new_columns = ['yr_built', 'bedrooms', 'yr_renovated', 'view', 'condition', 'grade', 'bathrooms', 'floors', 'waterfront', 'zipcode']
categorical.columns = new_columns
print(categorical.info())

At this point, I realized my dataset was growing, but without duplicate rows. My method of updating column information isn't working. The cell above this one wouldn't run at all and was causing kernal crashes.
The first number of rows that I have is 21597. Then after cleaning the dataset 23553. Then 1794 duplicated rows were deleted. At this point, the 21759 number of rows is still greater than the beginning number of rows in the data set. This is a problem. After updated the datatype for the bathrooms columns, my number of rows is equal to 23633, another increase. After changing the floors column, I had 114317 rows. In the next datatype update to categories for the column of data 'zipcode', the kernal dies. It is at this point, I realized that my method of updating information in the data is not working. So we begin again, with importing.

In [1]:
# Your code here - remember to use markdown cells for comments as well!
# import data set from repository
import pandas as pd
df = pd.read_csv('kc_house_data.csv', index_col=0)
df.head()

,date,price,bedrooms,bathrooms,sqft_living,sqft_lot,floors,waterfront,view,condition,grade,sqft_above,sqft_basement,yr_built,yr_renovated,zipcode,lat,long,sqft_living15,sqft_lot15
id,,,,,,,,,,,,,,,,,,,,
7129300520,10/13/2014,221900.0,3,1.00,1180,5650,1.0,NaN,0.0,3,7,1180,0.0,1955,0.0,98178,47.5112,-122.257,1340,5650
6414100192,12/9/2014,538000.0,3,2.25,2570,7242,2.0,0.0,0.0,3,7,2170,400.0,1951,1991.0,98125,47.7210,-122.319,1690,7639
5631500400,2/25/2015,180000.0,2,1.00,770,10000,1.0,0.0,0.0,3,6,770,0.0,1933,NaN,98028,47.7379,-122.233,2720,8062
2487200875,12/9/2014,604000.0,4,3.00,1960,5000,1.0,0.0,0.0,5,7,1050,910.0,1965,0.0,98136,47.5208,-122.393,1360,5000
1954400510,2/18/2015,510000.0,3,2.00,1680,8080,1.0,0.0,0.0,3,8,1680,0.0,1987,0.0,98074,47.6168,-122.045,1800,7503


In [2]:
# nans identification
print(len(df))
df.isna().sum()

21597


date                0
price               0
bedrooms            0
bathrooms           0
sqft_living         0
sqft_lot            0
floors              0
waterfront       2376
view               63
condition           0
grade               0
sqft_above          0
sqft_basement       0
yr_built            0
yr_renovated     3842
zipcode             0
lat                 0
long                0
sqft_living15       0
sqft_lot15          0
dtype: int64

In [3]:
# how much of the data is missing?
print('% of waterfront missing data: ' + str(df['waterfront'].isna().sum() / len(df['waterfront'])))
print('% of view missing data: ' + str(df['view'].isna().sum() / len(df['view'])))
print('% of yr_renovated data: ' + str(df['yr_renovated'].isna().sum() / len(df['yr_renovated'])))

% of waterfront missing data: 0.11001527989998611
% of view missing data: 0.0029170718155299346
% of yr_renovated data: 0.17789507802009538


In [5]:
# i realized that a potential solution was to look at the descibe method
df.describe()

,price,bedrooms,bathrooms,sqft_living,sqft_lot,floors,waterfront,view,condition,grade,sqft_above,yr_built,yr_renovated,zipcode,lat,long,sqft_living15,sqft_lot15
count,2.159700e+04,21597.000000,21597.000000,21597.000000,2.159700e+04,21597.000000,19221.000000,21534.000000,21597.000000,21597.000000,21597.000000,21597.000000,17755.000000,21597.000000,21597.000000,21597.000000,21597.000000,21597.000000
mean,5.402966e+05,3.373200,2.115826,2080.321850,1.509941e+04,1.494096,0.007596,0.233863,3.409825,7.657915,1788.596842,1970.999676,83.636778,98077.951845,47.560093,-122.213982,1986.620318,12758.283512
std,3.673681e+05,0.926299,0.768984,918.106125,4.141264e+04,0.539683,0.086825,0.765686,0.650546,1.173200,827.759761,29.375234,399.946414,53.513072,0.138552,0.140724,685.230472,27274.441950
min,7.800000e+04,1.000000,0.500000,370.000000,5.200000e+02,1.000000,0.000000,0.000000,1.000000,3.000000,370.000000,1900.000000,0.000000,98001.000000,47.155900,-122.519000,399.000000,651.000000
25%,3.220000e+05,3.000000,1.750000,1430.000000,5.040000e+03,1.000000,0.000000,0.000000,3.000000,7.000000,1190.000000,1951.000000,0.000000,98033.000000,47.471100,-122.328000,1490.000000,5100.000000
50%,4.500000e+05,3.000000,2.250000,1910.000000,7.618000e+03,1.500000,0.000000,0.000000,3.000000,7.000000,1560.000000,1975.000000,0.000000,98065.000000,47.571800,-122.231000,1840.000000,7620.000000
75%,6.450000e+05,4.000000,2.500000,2550.000000,1.068500e+04,2.000000,0.000000,0.000000,4.000000,8.000000,2210.000000,1997.000000,0.000000,98118.000000,47.678000,-122.125000,2360.000000,10083.000000
max,7.700000e+06,33.000000,8.000000,13540.000000,1.651359e+06,3.500000,1.000000,4.000000,5.000000,13.000000,9410.000000,2015.000000,2015.000000,98199.000000,47.777600,-121.315000,6210.000000,871200.000000


The following two cells will show a before and after of rewriting the nans portion of the code. I think the problem is with the type of merge that I'm doing, since the datasets are growing in length.

In [ ]:
# before
# managing nans
# fill waterfront and yr_renovated nans with unknown
waterfront_clean = pd.DataFrame(df.loc[:,'waterfront']).fillna(value='unknown')
yr_renovated_clean = pd.DataFrame(df.loc[:,'yr_renovated']).fillna(value='unknown')
df_clean = df_clean.drop(labels='waterfront', axis=1)
df_clean = df_clean.drop(labels='yr_renovated', axis=1)
df_clean = pd.merge(df_clean, waterfront_clean, left_index=True, right_index=True)
df_clean = pd.merge(df_clean, yr_renovated_clean, left_index=True, right_index=True)
# df_clean.head()

# delete nans rows in view
df_clean = df_clean.dropna()
print(len(df_clean))
df_clean.isna().sum()

In [ ]:
# after
# managing nans
# fill waterfront and yr_renovated nans with 'unknown'


In [54]:
continuous.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 21759 entries, 1000102 to 9900000190
Data columns (total 9 columns):
lat              21759 non-null float64
long             21759 non-null float64
sqft_lot         21759 non-null int64
sqft_living      21759 non-null int64
sqft_above       21759 non-null int64
sqft_basement    21759 non-null object
date             21759 non-null object
sqft_living15    21759 non-null int64
sqft_lot15       21759 non-null int64
dtypes: float64(2), int64(5), object(2)
memory usage: 1.7+ MB


In [53]:
target.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 21759 entries, 1000102 to 9900000190
Data columns (total 1 columns):
price    21759 non-null float64
dtypes: float64(1)
memory usage: 340.0 KB


### Explore Data

In [ ]:
# histograms
# normality assumptions (linear, normal, heterskedacity)
# heat maps
# correleations
# decisions

## Modeling

In [ ]:
# split into train and learn data sets
# multiple linear regression

## Interpret Model

## Evaluation of Model

In [ ]:
# use test data set here

## Deployment of Model